###### LightGBM version 6

In [ ]:


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('../input/lish-moa/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Read data

In [ ]:
folder_path = '/content/drive/My Drive/Colab Notebooks/'
input_path = folder_path #'../input/dictfeats/'
# param_path = '../input/params/'

In [ ]:

train = pd.read_csv(folder_path +'train_features.csv')

train.shape

(23814, 876)

In [ ]:
train_target = pd.read_csv(folder_path +'train_targets_scored.csv')

train_target.shape

(23814, 207)

In [ ]:
test = pd.read_csv(folder_path +'test_features.csv')

test.shape

## Check agonist / antagonist

In [ ]:
antagonists = []

for column in train_target.columns:
    if column.endswith('_agonist'):
        antagonist = column.replace('_agonist', '_antagonist')
        if antagonist in train_target.columns:
            antagonists.append((column, antagonist))

antagonists

[('acetylcholine_receptor_agonist', 'acetylcholine_receptor_antagonist'),
 ('adenosine_receptor_agonist', 'adenosine_receptor_antagonist'),
 ('adrenergic_receptor_agonist', 'adrenergic_receptor_antagonist'),
 ('androgen_receptor_agonist', 'androgen_receptor_antagonist'),
 ('cannabinoid_receptor_agonist', 'cannabinoid_receptor_antagonist'),
 ('dopamine_receptor_agonist', 'dopamine_receptor_antagonist'),
 ('estrogen_receptor_agonist', 'estrogen_receptor_antagonist'),
 ('gaba_receptor_agonist', 'gaba_receptor_antagonist'),
 ('glutamate_receptor_agonist', 'glutamate_receptor_antagonist'),
 ('histamine_receptor_agonist', 'histamine_receptor_antagonist'),
 ('opioid_receptor_agonist', 'opioid_receptor_antagonist'),
 ('ppar_receptor_agonist', 'ppar_receptor_antagonist'),
 ('progesterone_receptor_agonist', 'progesterone_receptor_antagonist'),
 ('retinoid_receptor_agonist', 'retinoid_receptor_antagonist'),
 ('serotonin_receptor_agonist', 'serotonin_receptor_antagonist'),
 ('sigma_receptor_agonis

List agonist / antagonist pairs where both features have 1

In [ ]:
for pair in antagonists:
    n = train_target[(train_target[pair[0]] == 1) & (train_target[pair[1]] == 1)].shape[0]
    if n > 0:
        print(pair[0], '-', pair[1])
        print('Number of cases:', n)

gaba_receptor_agonist - gaba_receptor_antagonist
Number of cases: 6
progesterone_receptor_agonist - progesterone_receptor_antagonist
Number of cases: 6
serotonin_receptor_agonist - serotonin_receptor_antagonist
Number of cases: 7
sigma_receptor_agonist - sigma_receptor_antagonist
Number of cases: 6


In [ ]:
train

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,...,c-60,c-61,c-62,c-63,c-64,c-65,c-66,c-67,c-68,c-69,c-70,c-71,c-72,c-73,c-74,c-75,c-76,c-77,c-78,c-79,c-80,c-81,c-82,c-83,c-84,c-85,c-86,c-87,c-88,c-89,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.1830,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,0.2198,...,0.4805,0.4965,0.3680,0.8427,0.1042,0.1403,0.1758,1.2570,-0.5979,1.2250,-0.0553,0.7351,0.5810,0.9590,0.2427,0.0495,0.4141,0.8432,0.6162,-0.7318,1.2120,0.6362,-0.4427,0.1288,1.4840,0.1799,0.5367,-0.1111,-1.0120,0.6685,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.1520,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,-0.2102,...,0.4083,0.0319,0.3905,0.7099,0.2912,0.4151,-0.2840,-0.3104,-0.6373,0.2887,-0.0765,0.2539,0.4443,0.5932,0.2031,0.7639,0.5499,-0.3322,-0.0977,0.4329,-0.2782,0.7827,0.5934,0.3402,0.1499,0.4420,0.9366,0.8193,-0.4236,0.3192,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,-0.4797,-0.5631,-0.0366,-1.8300,0.6057,-0.3278,0.6042,-0.3075,-0.1147,-0.0570,-0.0799,-0.8181,-1.5320,0.2307,0.4901,0.4780,-1.3970,4.6240,-0.0437,1.2870,-1.8530,0.6069,0.4290,0.1783,0.0018,-1.1800,...,-0.5477,-0.7576,-0.0444,0.1894,-0.0014,-2.3640,-0.4682,0.1210,-0.5177,-0.0604,0.1682,-0.4436,0.4963,0.1363,0.3335,0.9760,-0.0427,-0.1235,0.0959,0.0690,-0.9416,-0.7548,-0.1109,-0.6272,0.3019,0.1172,0.1093,-0.3113,0.3019,-0.0873,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,-0.8269,-0.3584,-0.8511,-0.5844,-2.5690,0.8183,-0.0532,-0.8554,0.1160,-2.3520,2.1200,-1.1580,-0.7191,-0.8004,-1.4670,-0.0107,-0.8995,0.2406,-0.2479,-1.0890,-0.7575,0.0881,-2.7370,0.8745,0.5787,-1.6740,...,-2.1220,-0.3752,-2.3820,-3.7350,-2.9740,-1.4930,-1.6600,-3.1660,0.2816,-0.2990,-1.1870,-0.5044,-1.7750,-1.6120,-0.9215,-1.0810,-3.0520,-3.4470,-2.7740,-1.8460,-0.5568,-3.3960,-2.9510,-1.1550,-3.2620,-1.5390,-2.4600,-0.9417,-1.5550,0.2431,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,-0.2219,-0.5121,-0.9577,1.1750,0.2042,0.1970,0.1244,-1.7090,-0.3543,-0.5160,-0.3330,-0.2685,0.7649,0.2057,1.3720,0.6835,0.8056,-0.3754,-1.2090,0.2965,-0.0712,0.6389,0.6674,-0.0783,1.1740,-0.7110,...,-0.2274,0.3215,0.1535,-0.4640,-0.5943,0.3973,0.1500,0.5178,0.5159,0.6091,0.1813,-0.4249,0.7832,0.6529,0.5648,0.4817,0.0587,0.5303,0.6376,-0.3966,-1.4950,-0.9625,-0.0541,0.6273,0.4563,0.0698,0.8134,0.1924,0.6054,-0.1824,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,0.3055,-0.4726,0.1269,0.2531,0.1730,-0.4532,-1.0790,0.2474,-0.4550,0.3588,0.1600,-0.7362,-0.1103,0.8550,-0.4139,0.5541,0.2310,-0.5573,-0.4397,-0.9260,-0.2424,-0.6686,0.2326,0.

In general agonist and antagonist should not have 1 at the same time

## Feature engineering

In [ ]:

train.at[train['cp_type'].str.contains('ctl_vehicle'),train.filter(regex='-.*').columns] = 0.0

test.at[test['cp_type'].str.contains('ctl_vehicle'),test.filter(regex='-.*').columns] = 0.0

## OHE

In [ ]:
train_size = train.shape[0]

traintest = pd.concat([train, test])

traintest = pd.concat([traintest, pd.get_dummies(traintest['cp_type'], prefix='cp_type')], axis=1)
traintest = pd.concat([traintest, pd.get_dummies(traintest['cp_time'], prefix='cp_time')], axis=1)
traintest = pd.concat([traintest, pd.get_dummies(traintest['cp_dose'], prefix='cp_dose')], axis=1)

traintest = traintest.drop(['cp_type', 'cp_time', 'cp_dose'], axis=1)

train = traintest[:train_size]
test  = traintest[train_size:]

del traintest

In [ ]:
train.shape

(23814, 880)

## Prepare x, y

In [ ]:
X_1 = train.drop('sig_id', axis=1)

Y_1 = train_target.drop('sig_id', axis=1)

X_final = test.drop('sig_id', axis=1)

## Feature selection

In [ ]:
# from https://www.kaggle.com/pavelvpster/moa-keras-optuna-rfecv

# 500
#selected_features = #['g-0', 'g-1', 'g-2', 'g-3', 'g-4', 'g-5', 'g-6', 'g-7', 'g-8', 'g-9', 'g-10', 'g-11', 'g-12', 'g-13', 'g-14', 'g-15', 'g-16', 'g-17', 'g-18', 'g-19', 'g-20', 'g-21', 'g-22', 'g-23', 'g-24', 'g-25', 'g-26', 'g-27', 'g-28', 'g-29', 'g-30', 'g-31', 'g-32', 'g-33', 'g-34', 'g-35', 'g-36', 'g-37', 'g-38', 'g-39', 'g-40', 'g-41', 'g-42', 'g-43', 'g-44', 'g-45', 'g-46', 'g-47', 'g-48', 'g-49', 'g-50', 'g-51', 'g-52', 'g-53', 'g-54', 'g-55', 'g-56', 'g-57', 'g-58', 'g-59', 'g-60', 'g-61', 'g-62', 'g-63', 'g-64', 'g-65', 'g-66', 'g-67', 'g-68', 'g-69', 'g-70', 'g-71', 'g-72', 'g-73', 'g-74', 'g-75', 'g-76', 'g-77', 'g-78', 'g-79', 'g-80', 'g-81', 'g-82', 'g-83', 'g-84', 'g-85', 'g-86', 'g-87', 'g-88', 'g-89', 'g-90', 'g-91', 'g-92', 'g-93', 'g-94', 'g-95', 'g-96', 'g-97', 'g-98', 'g-99', 'g-100', 'g-101', 'g-102', 'g-103', 'g-104', 'g-105', 'g-106', 'g-107', 'g-108', 'g-109', 'g-110', 'g-111', 'g-112', 'g-113', 'g-114', 'g-115', 'g-116', 'g-117', 'g-118', 'g-121', 'g-122', 'g-123', 'g-125', 'g-126', 'g-127', 'g-128', 'g-129', 'g-130', 'g-131', 'g-132', 'g-133', 'g-139', 'g-144', 'g-149', 'g-152', 'g-153', 'g-154', 'g-155', 'g-158', 'g-161', 'g-162', 'g-165', 'g-166', 'g-167', 'g-168', 'g-169', 'g-170', 'g-171', 'g-172', 'g-173', 'g-174', 'g-175', 'g-176', 'g-177', 'g-178', 'g-179', 'g-180', 'g-182', 'g-185', 'g-188', 'g-189', 'g-191', 'g-192', 'g-193', 'g-194', 'g-195', 'g-196', 'g-197', 'g-198', 'g-199', 'g-200', 'g-201', 'g-202', 'g-203', 'g-204', 'g-209', 'g-211', 'g-213', 'g-214', 'g-217', 'g-220', 'g-221', 'g-222', 'g-223', 'g-225', 'g-226', 'g-229', 'g-233', 'g-239', 'g-240', 'g-243', 'g-244', 'g-246', 'g-247', 'g-250', 'g-254', 'g-255', 'g-256', 'g-257', 'g-259', 'g-261', 'g-262', 'g-265', 'g-269', 'g-270', 'g-271', 'g-273', 'g-279', 'g-282', 'g-284', 'g-286', 'g-289', 'g-292', 'g-294', 'g-295', 'g-298', 'g-299', 'g-300', 'g-303', 'g-306', 'g-310', 'g-311', 'g-312', 'g-314', 'g-316', 'g-317', 'g-318', 'g-320', 'g-321', 'g-323', 'g-326', 'g-327', 'g-328', 'g-332', 'g-333', 'g-335', 'g-339', 'g-340', 'g-341', 'g-346', 'g-347', 'g-351', 'g-353', 'g-355', 'g-357', 'g-360', 'g-364', 'g-365', 'g-370', 'g-375', 'g-383', 'g-385', 'g-388', 'g-389', 'g-390', 'g-391', 'g-392', 'g-399', 'g-401', 'g-402', 'g-403', 'g-404', 'g-409', 'g-417', 'g-423', 'g-424', 'g-425', 'g-426', 'g-428', 'g-429', 'g-430', 'g-431', 'g-433', 'g-434', 'g-438', 'g-439', 'g-440', 'g-441', 'g-443', 'g-444', 'g-445', 'g-446', 'g-447', 'g-448', 'g-449', 'g-450', 'g-451', 'g-452', 'g-453', 'g-454', 'g-455', 'g-456', 'g-460', 'g-461', 'g-463', 'g-465', 'g-467', 'g-470', 'g-473', 'g-474', 'g-475', 'g-476', 'g-481', 'g-483', 'g-485', 'g-486', 'g-489', 'g-490', 'g-495', 'g-500', 'g-501', 'g-502', 'g-504', 'g-509', 'g-513', 'g-516', 'g-517', 'g-518', 'g-527', 'g-531', 'g-535', 'g-540', 'g-542', 'g-543', 'g-544', 'g-547', 'g-548', 'g-549', 'g-552', 'g-556', 'g-558', 'g-562', 'g-566', 'g-571', 'g-572', 'g-573', 'g-574', 'g-578', 'g-584', 'g-585', 'g-590', 'g-592', 'g-595', 'g-596', 'g-600', 'g-602', 'g-603', 'g-605', 'g-606', 'g-611', 'g-612', 'g-615', 'g-617', 'g-618', 'g-619', 'g-620', 'g-623', 'g-624', 'g-625', 'g-626', 'g-627', 'g-628', 'g-629', 'g-630', 'g-631', 'g-632', 'g-633', 'g-634', 'g-635', 'g-636', 'g-637', 'g-638', 'g-639', 'g-640', 'g-643', 'g-645', 'g-646', 'g-648', 'g-651', 'g-654', 'g-657', 'g-658', 'g-659', 'g-660', 'g-661', 'g-662', 'g-663', 'g-664', 'g-665', 'g-666', 'g-668', 'g-670', 'g-672', 'g-673', 'g-674', 'g-675', 'g-677', 'g-678', 'g-679', 'g-685', 'g-689', 'g-695', 'g-697', 'g-698', 'g-699', 'g-701', 'g-702', 'g-703', 'g-704', 'g-707', 'g-709', 'g-710', 'g-712', 'g-713', 'g-714', 'g-715', 'g-721', 'g-722', 'g-724', 'g-725', 'g-726', 'g-734', 'g-736', 'g-737', 'g-740', 'g-742', 'g-743', 'g-745', 'g-747', 'g-750', 'g-751', 'g-752', 'g-753', 'g-756', 'g-757', 'g-758', 'g-759', 'g-760', 'g-761', 'g-762', 'g-763', 'g-764', 'g-766', 'g-767', 'g-768', 'g-769', 'g-770', 'g-771', 'c-0', 'c-1', 'c-2', 'c-3', 'c-4', 'c-5', 'c-11', 'c-12', 'c-14', 'c-19', 'c-20', 'c-21', 'c-22', 'c-23', 'c-25', 'c-27', 'c-32', 'c-35', 'c-36', 'c-37', 'c-40', 'c-47', 'c-54', 'c-55', 'c-57', 'c-58', 'c-61', 'c-62', 'c-63', 'c-65', 'c-66', 'c-71', 'c-74', 'c-75', 'c-77', 'c-78', 'c-79', 'c-80', 'c-81', 'c-82', 'c-83', 'c-84', 'c-85', 'c-86', 'c-87', 'c-88', 'c-89', 'c-90', 'c-91', 'c-92', 'c-93', 'c-97']

In [ ]:
# x_train = x_train[selected_features]

# x_test = x_test[selected_features]

## LightGBM

In [ ]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
p_min = 0.001
p_max = 0.999
def logloss(y_true, y_pred):
    y_pred = np.clip(y_pred,p_min,p_max)
    return -np.mean(y_true*np.log(y_pred) + (1-y_true)*np.log(1-y_pred))

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_1,Y_1,test_size=0.3,random_state=9999)

In [ ]:
submission = pd.read_csv(folder_path +'sample_submission.csv')

In [ ]:
target_cols = ['dopamine_receptor_antagonist']


In [ ]:
import pickle
input_feats_file=input_path +'feats_important.txt'
with open(input_feats_file, 'rb') as handle:
  dict_feats = pickle.loads(handle.read())

In [ ]:
from sklearn.ensemble import VotingClassifier
params_xgb_1={
            'num_threads': 20,
            'random_state': 1000,
            'learning_rate': 0.01,
            'n_estimators':1000,
            'max_leaf_nodes': 100 ,
            'min_child_weight':0.0010123391323415569,
            'max_depth':10,
            'subsample': 0.7968351296815959,
            'colsample_bytree ': 0.8556374471450119,
            'alpha': 0.23497601594060086,
            'lambda': 0.15889208239516134
    }
params_xgb_2={
            'num_threads': 20,
            'random_state': 2000,
            'learning_rate': 0.01,
            'n_estimators':1000,
            'max_leaf_nodes': 100 ,
             'min_child_weight':0.0010123391323415569,
            'max_depth':10,
            'subsample': 0.7968351296815959,
            'colsample_bytree ': 0.8556374471450119,
            'alpha': 0.23497601594060086,
            'lambda': 0.15889208239516134
    }
params_lgb_1={
            'objective': 'binary',
            'metric': 'binary_logloss',
            'boosting_type': 'gbdt',
            'boost_from_average': True,
            'num_threads': 20,
            'random_state': 1000,

            'learning_rate': 0.01,
            'num_iterations':1000,

            # from Optuna result in Version 7
            'num_leaves': 100 ,
            'min_data_in_leaf': 92,
            'min_child_weight': 0.0010123391323415569,
            'max_depth': 10,
            'bagging_fraction': 0.7968351296815959,
            'feature_fraction': 0.8556374471450119,
            'lambda_l1': 0.23497601594060086,
            'lambda_l2': 0.15889208239516134
    }
params_lgb_2={
            'objective': 'binary',
            'metric': 'binary_logloss',
            'boosting_type': 'gbdt',
            'boost_from_average': True,
            'num_threads': 20,
            'random_state': 1000,

            'learning_rate': 0.01,
            'num_iterations':1000,

            # from Optuna result in Version 7
            'num_leaves': 100 ,
            'min_data_in_leaf': 92,
            'min_child_weight': 0.0010123391323415569,
            'max_depth': 10,
            'bagging_fraction': 0.7968351296815959,
            'feature_fraction': 0.8556374471450119,
            'lambda_l1': 0.23497601594060086,
            'lambda_l2': 0.15889208239516134
    }
params_gbc_1={
            'num_threads': 20,
            'random_state': 1000,
            'learning_rate': 0.01,
            'num_iterations':1000,
            # from Optuna result in Version 7
            'max_leaf_nodes': 100 ,
            'min_samples_leaf':92,
            'max_depth': 10,
            'min_samples_split': 0.7968351296815959
            
    }
params_gbc_2={
            'num_threads': 20,
            'random_state': 2000,

            'learning_rate': 0.01,
            'num_iterations':1000,

            # from Optuna result in Version 7
            'max_leaf_nodes': 100 ,
            'min_samples_leaf':92,
            'max_depth': 10,
            'min_samples_split': 0.7968351296815959
            
    }
xgb1 = xgb.XGBClassifier(params=params_xgb_1)
xgb2 = xgb.XGBClassifier(params=params_xgb_2)
lgb1 = lgb.LGBMClassifier(objective='binary',metric='binary_logloss',boosting_type='gbdt',boost_from_average=True,num_threads=20,random_state=1000,learning_rate=0.01,num_iterations=1000,num_leaves=100,min_data_in_leaf=92,min_child_weight=0.0010123391323415569,max_depth=10,bagging_fraction=0.7968351296815959,feature_fraction=0.8556374471450119,lambda_l1=0.23497601594060086,lambda_l2=0.15889208239516134)
lgb2 = lgb.LGBMClassifier(objective='binary',metric='binary_logloss',boosting_type='gbdt',boost_from_average=True,num_threads=20,random_state=2000,learning_rate=0.01,num_iterations=1000,num_leaves=100,min_data_in_leaf=92,min_child_weight=0.0010123391323415569,max_depth=10,bagging_fraction=0.7968351296815959,feature_fraction=0.8556374471450119,lambda_l1=0.23497601594060086,lambda_l2=0.15889208239516134)
gbc1 = GradientBoostingClassifier(random_state=2000,learning_rate=0.01,n_estimators=1000,max_leaf_nodes=100,min_samples_leaf=92,max_depth=10,min_samples_split=0.7968351296815959)
gbc2 = GradientBoostingClassifier(random_state=1000,learning_rate=0.01,n_estimators=1000,max_leaf_nodes=100,min_samples_leaf=92,max_depth=10,min_samples_split=0.7968351296815959)

#votingC = VotingClassifier(estimators=[('gbc1',gbc1),('gbc2',gbc2),('xgb1', xgb1),('xgb2',xgb2),('lgb1',lgb1),('lgb2',lgb2)], voting='soft')

votingC = VotingClassifier(estimators=[('lgb1',lgb1),('lgb2',lgb2)], voting='hard')

In [ ]:
def model_sub_1(X_train,y_train,X_test,y_test):
    model=votingC.fit(X_train,y_train)
    y_pred=model.predict_proba(X_test)
    score= logloss(y_test,y_pred[:,1])
    return model,score

In [ ]:
target_cols=Y_1.columns
target_cols=['dopamine_receptor_antagonist']

In [ ]:
scores = []
score_model={}
for col in target_cols:
    print(col)
    list_feats = dict_feats[col]
    X_train_res,y_train_res=X_train[list_feats],y_train[col]
    #X_train_res,y_train_res=X_train,y_train[col] #resampling_data(X_train,y_train[col])
    clf,score = model_sub_1(X_train_res,y_train_res,X_test[list_feats],y_test[col])
    test_pred=clf.predict_proba(X_final[list_feats])
    y_pred_final=test_pred[:,1]
    # test_pred=clf.predict(X_final[list_feats])
    # y_pred_final=test_pred
    print(score)
    score_model[col]=score
    submission[col]=y_pred_final
    scores.append(score)
print(np.mean(scores))

dopamine_receptor_antagonist


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.10632536442278681
0.10632536442278681


In [ ]:
print(score)

In [ ]:
sort_orders = sorted(score_model.items(), key=lambda x: x[1], reverse=True)

for i in sort_orders:
	print(i[0], i[1])


In [ ]:
# import matplotlib.pyplot as plt
# features = X_train.columns
# importances = clf.feature_importances_
# indices = np.argsort(importances)[::-1]


# for i in range(X_train.shape[1]):
#    print(features[indices[i]], importances[indices[i]]*100)

In [ ]:
submission.to_csv('submission.csv', index=False)